# Four point functions in the $O(n)$ model

This notebook reproduces some known results about Potts and $O(n)$ four-point functions, using the new Julia code.

In [1]:
Pkg.activate("."); # activate the parent environment
using BootstrapVirasoro, BootstrapVirasoro.LoopModels, BenchmarkTools
println("Number of threads: $(Threads.nthreads())")

  Activating 

Number of threads: 4


project at `~/Documents/Recherche/projet_these/code/BootstrapVirasoro/examples`


## Spectrum

We solve crossing symmetry equations for the spectrum of the $O(n)$ CFT:

\begin{align}
\mathcal{S}^{O(n)} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in 2\mathbb{N}+1} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \frac12\mathbb{N}^*\\ s\in\frac{1}{r}\mathbb{Z}}}  \ ,
\end{align}

## Bootstrap equations

We then numerically solve

\begin{align}
\sum_{V \in \mathcal{S}^{(s)}} D^{(s)}_V \mathcal I^{(s)}_V (x) = \sum_{V \in \mathcal{S}^{(t)}} D^{(t)}_V \mathcal I^{(t)}_V (x) = \sum_{V \in \mathcal{S}^{(u)}} D^{(u)}_V \mathcal I^{(u)}_V (x),
\end{align}

for some channel spectra $\mathcal{S}^{(s)}, \mathcal{S}^{(t)}, \mathcal{S}^{(u)}$, where $\mathcal I$ are interchiral conformal blocks, and the $D$'s are four-point structure constants.

We solve this system as
\begin{equation}
\underbrace{
\begin{bmatrix}
[\mathcal I^{(s)}_{V_j}(x_i)]_{ij} & [-\mathcal I^{(t)}_{V_j}(1-x_i)]_{ij} & [0] \\
[\mathcal I^{(s)}_{V_j}(x_i)]_{ij} & [0] & [-\mathcal I^{(u)}_{V_j}(1/x_i)]_{ij}
\end{bmatrix}}_A
\begin{bmatrix}
[D^{(s)}_{V_j}]_j \\
[D^{(t)}_{V_j}]_j \\
[D^{(u)}_{V_j}]_j
\end{bmatrix} = 
\begin{bmatrix}
\sum_{V_j \in \text{ known}} D^{(t)}_{V_j} \mathcal{I}^{(t)}_{V_j}(1-x_i) - \sum_{V_j \in \text{ known}} D^{(s)}_{V_j} \mathcal{I}^{(s)}_{V_j}(x_i)\\
\sum_{V_j \in \text{ known}} D^{(u)}_{V_j} \mathcal{I}^{(u)}_{V_j}(1/x_i) - \sum_{V_j \in \text{ known}} D^{(s)}_{V_j} \mathcal{I}^{(s)}_{V_j}(x_i)
\end{bmatrix}
\end{equation}
where the $x_i$ take more values than there are unknowns, i.e. $A$ is a tall rectangular matrix. 

To check numerical convergence, we solve two subsystems and compare the solutions. If the solutions are close, we know the computation has converged.


In [88]:
const Sig = Channels{Rational}

# Spectrum of O(n) model
c_S2 = CC(β=1 / (big"0.8" + big"0.1" * im))
ndiag_indices = [(r, s) for r in 1//2:1//2:20 for s in -1+1//(2r):1//(2r):1 if r * s % 1 == 0]
diag_field_S2 = Field(c_S2, r=0, s=big"0.4" + big"0.1" * im)
fields_S2 = vcat([Field(c_S2, r=r, s=s) for (r, s) in ndiag_indices], diag_field_S2)

# Determine the parity of the number of legs in 4pt channels
function chan_parities(co::Correlation4)
    V1, V2, V3, V4 = co.fields
    chan_parities = Channels{Rational}((V1.r + V2.r) % 1, (V1.r + V4.r) % 1, (V1.r + V3.r) % 1)
end

# Compute series of blocks for any channel field compatible
# with the correlation co
function LoopSpectra(co, fields, fs)
    Vs = @channels filter(V -> V.r % 1 == chan_parities(co)[chan], fields)
    @channels ChannelSpectrum(co, chan, Vs[chan], fs[chan])
end

# Compute series of blocks, keeping only one of each (field, reflected field) pair
# given indices of the external fields.
function precompute_blocks(indices, fields; parity, precision=10)
    setprecision(BigFloat, floor(Int, 1.2 * precision), base=10)
    Δmax = floor(Int, 1.5 * precision)
    co = Correlation([Field(c_S2, r=r, s=s) for (r, s) in ind], Δmax)
    parity != 0 && (fields = filter(V -> V.diagonal || V.s >= 0, fields))
    fs = Channels(chan -> (V -> IBlock(co, chan, V, parity=parity)))
    LoopSpectra(co, fields, fs)
end

# Solve crossing symmetry for given signature
function solve(specs, signature; even_spin=Channels(false), int_spin=Channels(false), rmax=3, show=true)
    specs = @channels filter(V -> V.r >= signature[chan], specs[chan])
    specs = @channels filter(V -> even_spin[chan] ? spin(V) % 2 == 0 : true, specs[chan])
    specs = @channels filter(V -> int_spin[chan] ? spin(V) % 1 == 0 : true, specs[chan])
    system = solve_bootstrap(specs)
    if show
        Base.show(stdout, system.str_cst, rmax=rmax)
    end
    return system
end

solve (generic function with 2 methods)

## $\langle (\frac{1}{2}, 0)^4 \rangle$

In [7]:
ind = ((1 // 2, 0), (1 // 2, 0), (1 // 2, 0), (1 // 2, 0))
blocks_even = precompute_blocks(ind, fields_S2, parity=1, precision=10);

In [8]:
sol = solve(blocks_even, Sig(0, 1, 1), rmax=2);

 Channel s
┌─────────────────┬────────────────────────────┬────────────────┐
│ Field           │ Structure constant         │ Relative error │
├─────────────────┼────────────────────────────┼────────────────┤
│ (P=0.16+0.06im) │                  1.0+0.0im │            0.0 │
│          (1, 0) │       -0.255839+0.565057im │     1.53701e-8 │
│          (1, 1) │                          0 │       0.849788 │
│          (2, 0) │  0.000304229+0.000902009im │     6.79824e-6 │
│       (2, 1//2) │                          0 │       0.246734 │
│          (2, 1) │ -8.36374e-05-0.000396747im │     1.05754e-5 │
└─────────────────┴────────────────────────────┴────────────────┘
 Channel t
┌───────────┬────────────────────────────┬────────────────┐
│ Field     │ Structure constant         │ Relative error │
├───────────┼────────────────────────────┼────────────────┤
│    (1, 0) │       -0.255839+0.565057im │     1.53701e-8 │
│    (1, 1) │                          0 │       0.849788 │
│    (2, 0) │  0.0

Benchmarks: on my 2015, 4-core Intel Macbook pro.
| Precision          | Python | Julia |
|----------|----------|----------|
| $\Delta_{\mathrm{max}}=20$, 13 digits  | 2min23s  | 2.3s  |
| $\Delta_{\mathrm{max}}=30$, 25 digits  | 8min10s  | 3.0s  |
| $\Delta_{\mathrm{max}}=40$, 35 digits  | 23min18s  | 12.6s  |

## $\langle (\frac{1}{2}, 0)^2 (1, 0)^2 \rangle$

In [5]:
ind = ((1//2, 0), (1//2, 0), (1, 0), (1, 0))
blocks_even = precompute_blocks(ind, fields_S2, parity=1, precision=20);

### Signature $(0, \frac32, \frac32)$

In [6]:
sol = solve(blocks_even, Sig(0, 3//2, 3//2));

 Channel s
┌─────────────────┬────────────────────────────┬────────────────┐
│ Field           │ Structure constant         │ Relative error │
├─────────────────┼────────────────────────────┼────────────────┤
│ (P=0.16+0.06im) │                  1.0+0.0im │            0.0 │
│          (1, 0) │        -0.26148+0.923384im │    6.51652e-19 │
│          (1, 1) │                          0 │        2.89824 │
│          (2, 0) │       0.0017207-0.002615im │    3.53625e-16 │
│       (2, 1//2) │                          0 │        3.81398 │
│          (2, 1) │                          0 │        3.17457 │
│          (3, 0) │ -1.28106e-06-8.33233e-08im │    3.05527e-12 │
│       (3, 1//3) │                          0 │        6.84996 │
│       (3, 2//3) │  8.01481e-07-6.16705e-08im │    1.55871e-12 │
│          (3, 1) │                          0 │        2.83822 │
└─────────────────┴────────────────────────────┴────────────────┘
 Channel t
┌──────────────┬──────────────────────┬───────────────

### Signature $(1, \frac12, \frac32)$

In [80]:
solve(blocks_even, Sig(1, 1//2, 3//2));

 Channel s
┌───────────┬────────────────────────────┬────────────────┐
│ Field     │ Structure constant         │ Relative error │
├───────────┼────────────────────────────┼────────────────┤
│    (1, 0) │                  1.0+0.0im │            0.0 │
│    (1, 1) │       0.502183-0.0830317im │    9.88134e-19 │
│    (2, 0) │     0.00239155+0.0255139im │    6.66122e-17 │
│ (2, 1//2) │     0.00273704+0.0200265im │    2.02014e-17 │
│    (2, 1) │                          0 │        1.28257 │
│    (3, 0) │ -6.40312e-07-1.92164e-06im │     2.5215e-12 │
│ (3, 1//3) │ -7.07647e-07-9.43683e-07im │    3.51888e-12 │
│ (3, 2//3) │ -6.00257e-07+2.35486e-07im │    2.48351e-12 │
│    (3, 1) │                          0 │         1.4421 │
└───────────┴────────────────────────────┴────────────────┘
 Channel t
┌──────────────┬────────────────────────────┬────────────────┐
│ Field        │ Structure constant         │ Relative error │
├──────────────┼────────────────────────────┼────────────────┤
│    (1//

### Signature $(1, \frac32, \frac12)$

In [81]:
sol = solve(blocks_even, Sig(1, 3//2, 1//2));

 Channel s
┌───────────┬────────────────────────────┬────────────────┐
│ Field     │ Structure constant         │ Relative error │
├───────────┼────────────────────────────┼────────────────┤
│    (1, 0) │                  1.0+0.0im │            0.0 │
│    (1, 1) │      -0.502183+0.0830317im │    6.14357e-19 │
│    (2, 0) │     0.00239155+0.0255139im │     4.3717e-17 │
│ (2, 1//2) │    -0.00273704-0.0200265im │    1.22157e-17 │
│    (2, 1) │                          0 │        1.38137 │
│    (3, 0) │ -6.40312e-07-1.92164e-06im │     2.3246e-12 │
│ (3, 1//3) │  7.07647e-07+9.43683e-07im │     2.8984e-12 │
│ (3, 2//3) │ -6.00257e-07+2.35486e-07im │    2.18162e-12 │
│    (3, 1) │                          0 │        1.29062 │
└───────────┴────────────────────────────┴────────────────┘
 Channel t
┌──────────────┬────────────────────────────┬────────────────┐
│ Field        │ Structure constant         │ Relative error │
├──────────────┼────────────────────────────┼────────────────┤
│    (3//

## $\left\langle (0, \frac12)^3 P \right\rangle$

In [ ]:
ind = ((0, 1//2), (0, big"0.2"+big"0.4"*im), (0, 1//2), (0, 1//2))
blocks_even_S2 = precompute_blocks(ind, fields_S2, parity=1, precision=20);
solve(blocks_even_S2, Sig(0, 0, 0), Channels(true), rmax=4);

 Channel s
┌─────────────────┬────────────────────────────┬────────────────┐
│ Field           │ Structure constant         │ Relative error │
├─────────────────┼────────────────────────────┼────────────────┤
│ (P=0.16+0.06im) │                  1.0+0.0im │            0.0 │
│          (1, 0) │       0.033039+0.0170519im │    1.53044e-19 │
│          (2, 0) │  2.18931e-05+8.64717e-06im │    2.31071e-16 │
│          (2, 1) │  1.81338e-05-1.64188e-05im │    4.27661e-17 │
│          (3, 0) │ -3.15216e-11+1.70895e-10im │    2.43722e-10 │
│       (3, 2//3) │  -1.5888e-10+8.58888e-10im │    5.21382e-12 │
│          (4, 0) │ -1.43798e-16-1.12514e-16im │     0.00208119 │
│       (4, 1//2) │ -2.78536e-16-1.98927e-16im │    0.000460803 │
│          (4, 1) │    6.0737e-17-1.7951e-16im │    0.000379093 │
└─────────────────┴────────────────────────────┴────────────────┘
 Channel t
┌─────────────────┬────────────────────────────┬────────────────┐
│ Field           │ Structure constant         │ Relat

## $\left\langle (0, \frac12)^3 (1, 0) \right\rangle$

In [89]:
ind = ((1, 0), (0, 1//2), (0, 1//2), (0, 1//2))
blocks_even_S2 = precompute_blocks(ind, fields_S2, parity=1, precision=20);

In [ ]:
sys_S2 = solve(blocks_even_S2, Sig(0, 1, 1), even_spin=Channels(true, false, false), rmax=2);

 Channel s
┌─────────────────┬────────────────────────────┬────────────────┐
│ Field           │ Structure constant         │ Relative error │
├─────────────────┼────────────────────────────┼────────────────┤
│ (P=0.16+0.06im) │                  1.0+0.0im │            0.0 │
│          (1, 0) │                          0 │        12.0715 │
│          (2, 0) │  -0.000314628+0.00161483im │    5.58133e-17 │
│          (2, 1) │ -1.68652e-05-0.000641317im │    2.21923e-17 │
└─────────────────┴────────────────────────────┴────────────────┘
 Channel t
┌───────────┬────────────────────────────┬────────────────┐
│ Field     │ Structure constant         │ Relative error │
├───────────┼────────────────────────────┼────────────────┤
│    (1, 0) │                          0 │        12.0715 │
│    (1, 1) │  -0.000314628+0.00161483im │    5.58133e-17 │
│    (2, 0) │ -1.68652e-05-0.000641317im │    2.21923e-17 │
│ (2, 1//2) │                          0 │       0.341744 │
│    (2, 1) │                 

## $\left\langle (0, \frac12)^3 (2, 0) \right\rangle$

In [31]:
ind = ((0, 1//2), (2, 0), (0, 1//2), (0, 1//2))
blocks_even_S2 = precompute_blocks(ind, fields_S2, parity=1, precision=20);
solve(blocks_even_S2, Sig(1, 2, 1), Channels(false, true, false), rmax=4);

 Channel s
┌───────────┬────────────────────────────┬────────────────┐
│ Field     │ Structure constant         │ Relative error │
├───────────┼────────────────────────────┼────────────────┤
│    (1, 0) │                  1.0+0.0im │            0.0 │
│    (1, 1) │       -0.633084+0.088108im │    5.07475e-20 │
│    (2, 0) │                          0 │       0.874759 │
│ (2, 1//2) │                          0 │       0.549149 │
│    (2, 1) │                          0 │       0.869633 │
│    (3, 0) │ -6.59937e-05+5.41536e-05im │    1.68373e-13 │
│ (3, 1//3) │  2.79701e-05-2.44819e-05im │    3.67846e-13 │
│ (3, 2//3) │  1.83437e-05-1.91256e-05im │    3.77195e-13 │
│    (3, 1) │ -1.79552e-05+2.51328e-05im │    1.87836e-13 │
│    (4, 0) │                          0 │        1.49792 │
│ (4, 1//4) │                          0 │        1.49717 │
│ (4, 1//2) │                          0 │        1.49586 │
│ (4, 3//4) │                          0 │        1.49431 │
│    (4, 1) │                

## $\left\langle (0, \frac12)^3 (3, 0) \right\rangle$

In [24]:
ind = ((0, 1//2), (3, 0), (0, 1//2), (0, 1//2))
blocks_even_S2 = precompute_blocks(ind, fields_S2, parity=1, precision=20);
solve(blocks_even_S2, Sig(2, 2, 2), Channels(true), rmax=4);

 Channel s
┌───────────┬────────────────────────────┬────────────────┐
│ Field     │ Structure constant         │ Relative error │
├───────────┼────────────────────────────┼────────────────┤
│    (2, 0) │                  1.0+0.0im │            0.0 │
│    (2, 1) │      -0.460849+0.0936033im │    2.67895e-16 │
│    (3, 0) │                          0 │         1.8295 │
│ (3, 2//3) │                          0 │        6.62341 │
│    (4, 0) │ -1.37046e-06-3.46191e-07im │      6.0413e-9 │
│ (4, 1//2) │  1.02402e-06+1.79868e-07im │     6.24716e-9 │
│    (4, 1) │  -4.72437e-07+1.6266e-08im │     7.20866e-9 │
└───────────┴────────────────────────────┴────────────────┘
 Channel t
┌───────────┬────────────────────────────┬────────────────┐
│ Field     │ Structure constant         │ Relative error │
├───────────┼────────────────────────────┼────────────────┤
│    (2, 0) │      -0.460849+0.0936033im │    2.67895e-16 │
│    (2, 1) │                          0 │         1.8295 │
│    (3, 0) │     

# Torus one point functions

In [83]:
import EllipticFunctions: etaDedekind
import BootstrapVirasoro: PosCache, xfromq, qfromx
const Sig = Channels{Rational}

c = CC(β=sqrt(big"2")/(big"0.8"+big"0.1"*im))
ndiag_indices = [(r, s) for r in 1//2:1//2:20 for s in -1+1//(2r):1//(2r):1 if (r * s) % (1//2) == 0]
diag_field = Field(c, r=0, s=big"0.4"+big"0.1"*im)
fields = vcat([Field(c, r = r, s = s) for (r, s) in ndiag_indices], diag_field)

function precompute_blocks(index::Tuple{<:Number, <:Number}, fields; parity, precision=10)
    setprecision(BigFloat, floor(Int, 1.2*precision), base=10)
    Δmax = floor(Int, 0.75*precision)
    co = Correlation(Field(c, r=index[1], s=index[2]), Δmax)
    parity != 0 && (fields = filter(V -> V.diagonal || V.s >= 0, fields))
    f = V -> IBlock(co, V, parity=parity)
    spec = ChanSpec(co, fields, f)
    return @channels spec
end

xfromτ(τ) = xfromq(exp(im * (π * τ)))

function prefA(τ, co_s, chan, lr)
    x_cache = PosCache(xfromτ(τ), co_s[lr], chan)
    return inv(
        etaDedekind(τ) * x_cache.prefactor
    )
end

ncprefA(τ, co_s, chan) = prefA(τ, co_s, chan, :left) * 
    prefA(-conj(τ), co_s, chan, :right)

relative_error(a, b) = abs((a - b) / (a + b))

relative_error (generic function with 1 method)

## r = 0

In [84]:
ind = (0, sqrt(big"2")*(big"0.4"+big"0.2"*im))
blocks = precompute_blocks(ind, fields, parity=1, precision=20);

In [85]:
solve(blocks, Sig(0, 0, 0), rmax=2);

 Channel s
┌─────────────────┬────────────────────────┬────────────────┐
│ Field           │ Structure constant     │ Relative error │
├─────────────────┼────────────────────────┼────────────────┤
│ (P=0.11+0.04im) │              1.0+0.0im │            0.0 │
│       (1//2, 0) │   0.308727-0.0524853im │    9.81208e-18 │
│       (1//2, 1) │                      0 │        1.07228 │
│          (1, 0) │   0.0780102-0.123407im │    8.81193e-15 │
│       (1, 1//2) │                      0 │        1.04229 │
│          (1, 1) │   -0.118635-0.139376im │    3.30164e-15 │
│       (3//2, 0) │ 0.00494416-0.0354923im │     5.14698e-8 │
│    (3//2, 1//3) │                      0 │       0.915309 │
│    (3//2, 2//3) │  -0.144948-0.0687758im │     1.12038e-8 │
│       (3//2, 1) │                      0 │       0.939375 │
│          (2, 0) │     1.52682+0.262524im │        1.06699 │
│       (2, 1//4) │    -1.52644-0.237779im │        1.05741 │
│       (2, 1//2) │     1.45133+0.330688im │        1.10473

## r = 1

In [86]:
blocks = precompute_blocks((1, 0), fields, parity=1, precision=20);

In [ ]:
sys = solve(blocks, Sig(0, 1//2, 1//2), rmax=3//2);

 Channel s
┌─────────────────┬─────────────────────┬────────────────┐
│ Field           │ Structure constant  │ Relative error │
├─────────────────┼─────────────────────┼────────────────┤
│ (P=0.11+0.04im) │           1.0+0.0im │            0.0 │
│       (1//2, 0) │                   0 │         1.2836 │
│       (1//2, 1) │                   0 │       0.556289 │
│          (1, 0) │   5.18288-2.43916im │    1.74818e-16 │
│       (1, 1//2) │                   0 │       0.764127 │
│          (1, 1) │  -4.84733+2.26298im │    7.86559e-17 │
│       (3//2, 0) │                   0 │        1.00824 │
│    (3//2, 1//3) │                   0 │       0.999661 │
│    (3//2, 2//3) │                   0 │       0.975496 │
│       (3//2, 1) │                   0 │       0.964301 │
│          (2, 0) │   10.9059-9.71077im │       0.128221 │
│       (2, 1//4) │ 0.195797+0.109164im │        1.05359 │
│       (2, 1//2) │  -2.16255+4.97321im │       0.243669 │
│       (2, 3//4) │ 0.197122+0.106252im │    

## r = 2

In [205]:
blocks_even = precompute_blocks((2, 0), fields, parity=1, precision=25);

In [206]:
sys = solve(blocks_even, Sig(1//2, 1//2, 1));

Channel s
┌──────────────┬────────────────────────────┬────────────────┐
│ Field        │ Structure constant         │ Relative error │
├──────────────┼────────────────────────────┼────────────────┤
│    (1//2, 0) │                  1.0+0.0im │            0.0 │
│       (1, 0) │         -370.247-6589.17im │       0.346136 │
│       (1, 1) │         -551.224-4312.16im │        0.35027 │
│    (3//2, 0) │  6.14453e+06-2.00515e+06im │       0.379677 │
│ (3//2, 2//3) │  5.83822e+06-1.87505e+06im │       0.374344 │
│       (2, 0) │  3.13349e+10+1.63396e+10im │       0.436682 │
│    (2, 1//2) │  3.75186e+10+1.60223e+10im │       0.354676 │
│       (2, 1) │  2.95116e+10+1.50891e+10im │       0.428881 │
│    (5//2, 0) │  1.44027e+15+1.01898e+15im │       0.457327 │
│ (5//2, 2//5) │   1.71854e+15+1.0154e+15im │       0.347922 │
│ (5//2, 4//5) │  1.51624e+15+9.93452e+14im │        0.40509 │
│       (3, 0) │  1.01213e+22-1.62988e+22im │       0.642458 │
│    (3, 1//3) │  -8.4379e+21+1.63741e+22im │

## r = 3

In [70]:
blocks = precompute_blocks((3, 0), fields, parity=1, precision=20);

In [71]:
sys = solve(blocks, Sig(1, 1, 1), rmax=2);

 Channel s
┌──────────────┬──────────────────────┬────────────────┐
│ Field        │ Structure constant   │ Relative error │
├──────────────┼──────────────────────┼────────────────┤
│       (1, 0) │            1.0+0.0im │            0.0 │
│       (1, 1) │ -1.12599-0.0241255im │    2.15821e-20 │
│    (3//2, 0) │                    0 │        1.62435 │
│ (3//2, 2//3) │                    0 │        1.37079 │
│       (2, 0) │   -81580.2-19167.9im │    1.18339e-11 │
│    (2, 1//2) │    74953.0+16184.7im │    8.35654e-12 │
│       (2, 1) │   -66144.5-11403.8im │    6.65419e-12 │
└──────────────┴──────────────────────┴────────────────┘
 Channel t
┌──────────────┬──────────────────────┬────────────────┐
│ Field        │ Structure constant   │ Relative error │
├──────────────┼──────────────────────┼────────────────┤
│       (1, 0) │ -1.12599-0.0241255im │    2.15821e-20 │
│       (1, 1) │                    0 │        1.62435 │
│    (3//2, 0) │                    0 │        1.37079 │
│ (3//2, 

## r = 4